# Лабораторная работа №3


### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [ ]:
t = "бобоба" #подстрока

p = [0]*len(t)
j = 0
i = 1

while i < len(t):
    if t[j] == t[i]:
        p[i] = j+1
        i += 1
        j += 1
    else:
        if j == 0:
            p[i] = 0
            i += 1
        else:
            j = p[j-1]

print(p)

a = "бобобесь бобобась" #строка
m = len(t)
n = len(a)

i = 0
j = 0
while i < n:
    if a[i] == t[j]:
        i += 1
        j += 1
        if j == m:
            print("Подстрока найдена")
            break
    else:
        if j > 0:
            j = p[j-1]
        else:
            i += 1

if i == n and j != m:
    print("Подстрока не найден")

#### Упрощенный алгоритм Бойера-Мура

In [5]:
obraz = "Зорро" 
stroka = "Ещё в 17 веке в Испании был известен персонаж Зорро"

#Таблица смещений 
def table_s(str1):
    S = set()
    symbols = {}
    for i in range (len(obraz) - 2, -1, -1):
        if obraz[i] not in S:
            symbols[obraz[i]] = len(obraz) - i - 1
            S.add(obraz[i])

    if obraz[len(obraz) - 1] not in S:
        symbols[obraz[len(obraz) - 1]] = len(obraz)

    symbols['*'] = len(obraz)

    return symbols

print(table_s(obraz))

def murSearch(str1, x):
    d = table_s(x)
    i = len(x)-1
    lenObraz = j = k = len(x) - 1

    while j > 0 and i <= len(str1):
        if str1[k-1] == x[j-1]:
            k -= 1
            j -= 1
        else:
            if d.get(str1[k]) != None:
                i += d[str1[k]]
            else:
                i += d['*']
            j = lenObraz
            k = i
    
    if j <=0:
        return f"Индекс найденной подстроки: {k}"
    return "Строка не найдена "

print(murSearch(stroka, obraz))

{'р': 1, 'о': 3, 'З': 4, '*': 5}
Индекс найденной подстроки: 46


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [ ]:
from queue import PriorityQueue

class Position: 
    def __init__ (self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__ (self, other):
        return  other.start_distance > self.start_distance

    def __str__ (self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

N = 4

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)
    if i < N - 1: offsets.append(N)
    if j > 0: offsets.append(-1)
    if j < N - 1: offsets.append(1)
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)  
                i = permutation[i]
    return (cycles + len(permutation)) % 2

SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3,  4, 5, 6, 7, 0,8, 13, 9, 11, 12, 10, 14, 15]

if parity(start) == 0:
    print('Неверно')
else:
    start = tuple(start)
    p = Position(start, 0)
    print(p, '\n')

    candidates = PriorityQueue()
    candidates.put(p)

    visited = set([p])
    care_from = {p.position: None}

    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                care_from[k] = p
                visited.add(k)

    path = []
    prev = p
    while p.position != start:
        p = care_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  0
  8 13  9 11
 12 10 14 15 



### Вывод